# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
spacy_udpipe.download('sk')
nlp = spacy_udpipe.load('sk')

Already downloaded a model for the 'sk' language


In [3]:
# set variables 
LANG = 'sk'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('a', 'a'),
 ('v', 'v'),
 ('na', 'na'),
 ('sa', 'sa'),
 ('je', 'byť'),
 ('s', 's'),
 ('align', 'align'),
 ('right', 'right'),
 ('z', 'z'),
 ('aj', 'aj'),
 ('o', 'o'),
 ('do', 'do'),
 ('to', 'to'),
 ('ako', 'ako'),
 ('že', 'že'),
 ('pre', 'pre'),
 ('si', 'si'),
 ('V', 'v'),
 ('roku', 'rok'),
 ('za', 'za'),
 ('alebo', 'alebo'),
 ('sú', 'byť'),
 ('som', 'byť'),
 ('od', 'od'),
 ('k', 'k'),
 ('po', 'po'),
 ('ale', 'ale'),
 ('ktoré', 'ktorý'),
 ('bol', 'byť'),
 ('ktorý', 'ktorý'),
 ('tak', 'tak'),
 ('pri', 'pr'),
 ('vo', 'v'),
 ('by', 'by'),
 ('už', 'už'),
 ('čo', 'čo'),
 ('ktorá', 'ktorý'),
 ('bola', 'byť'),
 ('jeho', 'jeho'),
 ('má', 'mať'),
 ('len', 'len'),
 ('ich', 'on'),
 ('byť', 'byť'),
 ('nie', 'nie'),
 ('MBA', 'mba'),
 ('_', '_'),
 ('Na', 'na'),
 ('až', 'až'),
 ('aby', 'aby'),
 ('podľa', 'podľa')]


In [6]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

919919
1525977


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [7]:
model_big["zlúčenina"]

array([ 0.03  , -0.0308,  0.0151,  0.0845, -0.0559,  0.0118,  0.0218,
       -0.024 ,  0.0429,  0.0446, -0.0231,  0.0223,  0.039 , -0.0432,
       -0.0284,  0.1014,  0.0681,  0.048 ,  0.0295,  0.0434, -0.0427,
       -0.021 ,  0.014 ,  0.001 ,  0.0552,  0.0079,  0.0088, -0.011 ,
       -0.0047, -0.0562,  0.0104,  0.0114,  0.0392, -0.0123,  0.0065,
        0.0141,  0.0328,  0.0027,  0.0175, -0.0314, -0.0113, -0.0229,
       -0.0334,  0.0347, -0.0754,  0.0432, -0.0014,  0.0389, -0.0004,
       -0.0316,  0.015 , -0.0051, -0.0136,  0.0191,  0.0845, -0.0409,
       -0.0242, -0.0378,  0.0293,  0.0466,  0.0163, -0.0351,  0.0283,
        0.0686, -0.0942, -0.0243, -0.046 , -0.02  , -0.0033,  0.0149,
       -0.0051,  0.0478, -0.0125, -0.0326,  0.0169, -0.026 ,  0.0204,
       -0.0606, -0.0033,  0.0029,  0.027 ,  0.0081,  0.0703,  0.0579,
        0.0111, -0.0095, -0.0805, -0.0123, -0.0951,  0.0571,  0.0228,
        0.0261, -0.0052,  0.0378,  0.0076,  0.0091,  0.0198,  0.0271,
       -0.0215,  0.0

In [8]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('byť', 117),
 ('mať', 115),
 ('on', 71),
 ('vedieť', 70),
 ('ísť', 66),
 ('hrať', 66),
 ('robiť', 65),
 ('poznať', 65),
 ('a', 64),
 ('hľadať', 61),
 ('to', 60),
 ('chcieť', 59),
 ('urobiť', 59),
 ('môcť', 58),
 ('musieť', 58),
 ('starý', 58),
 ('ja', 56),
 ('veľký', 56),
 ('veriť', 55),
 ('pozerať', 55)]

In [9]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


144748 144748


In [10]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [11]:
# test the model
pprint(new_vector_space.most_similar('otec'))
pprint(new_vector_space.most_similar_cosmul(positive=['žena', 'král'], negative=['muž']))

[('syn', 0.7990519404411316),
 ('otcov', 0.7085798978805542),
 ('matka', 0.7008340358734131),
 ('syna', 0.6916787624359131),
 ('brat', 0.6684491634368896),
 ('mama', 0.6567193269729614),
 ('otcova', 0.6508373022079468),
 ('ded', 0.6416566371917725),
 ('rodič', 0.6386483907699585),
 ('otcovstvo', 0.6213869452476501)]
[('královna', 0.8618711233139038),
 ('královná', 0.8517712950706482),
 ('králový', 0.8510134816169739),
 ('králů', 0.8508755564689636),
 ('králík', 0.8334306478500366),
 ('královský', 0.8136569857597351),
 ('královo', 0.804227888584137),
 ('kráľovná', 0.7996811270713806),
 ('královstvie', 0.7994561791419983),
 ('krála', 0.7991824746131897)]


In [12]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [13]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  2%|▏         | 91/5242 [00:00<00:05, 904.86it/s]

5242


100%|██████████| 5242/5242 [00:04<00:00, 1235.09it/s]

5242


In [15]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1")
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2")
        elif word_nolemma in words_lower:
            found = 1
            print("w3")

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

l2 potravový
w1
l3 vôd
w3
l3 dostatočné
w2
112 needs to be added manually:  salinizácia  -  salinizácia  -  salinizácia pôdy
l2 fyzikálne
l2 živočí
l2 ľudia
l2 mliekárska
l3 pôdne
l2 priesakový
296 needs to be added manually:  prevlhčený  -  prevlhčená  -  zamokrená prevlhčená pôda
w2
l2 federálne
l2 cementárska
w1
w1
423 needs to be added manually:  bezzávlahový  -  bezzávlahové  -  bezzávlahové hospodárenie
l3 pôdne
l2 vrstevnicie
w3
l2 geofyzikálne
l2 biogeografický
l2 cukrovarnícka
w3
l2 živočí
660 needs to be added manually:  biofond  -  biofond  -  biologické zdroje biofond
l2 patrimoniálny
w1
l2 zasahovaný
l2 veľkochova
l2 porážanie
l2 protierózny
l2 ovzdušiť
l2 sekundárne
l2 aditívny
861 needs to be added manually:  máloprodukčný  -  máloprodukčná  -  máloprodukčná pôda
l2 demineralizácia
l2 fyzikálne
l2 litorálna
w1
l3 pôdne
w2
w1
l2 nerozložiteľný
1023 needs to be added manually:  parcelovanie  -  parcelovanie  -  parcelovanie pôdy
l2 terasovit
l2 živočí
w1
l3 proti
l2 antise

l2 imisný
l2 zvyška
l2 emulgácia
l2 chlorovodíkovo
l2 azbestóz
4565 needs to be added manually:  geogénna  -  geogénny  -  geogénny faktor
4569 needs to be added manually:  želežný  -  želežný  -  želežný šrot
l2 vyhnívanie
4575 needs to be added manually:  tenatový  -  tenatových  -  lov do tenatových sietí
l2 flotácia
l2 koliformný
4604 needs to be added manually:  schistosomiáza  -  schistosomiáza  -  schistosomiáza
4615 needs to be added manually:  zneškodniteľný  -  zneškodniteľné  -  ťažko zneškodniteľné odpady
4617 needs to be added manually:  dekantácia  -  dekantácia  -  dekantácia
l2 hydrometrie
4626 needs to be added manually:  biokoncentrácia  -  biokoncentrácie  -  faktor biokoncentrácie
l2 halogénovaný
l2 návrš
4669 needs to be added manually:  halón  -  halónov  -  zákaz freónov a halónov
l2 teratogénny
4692 needs to be added manually:  dekantácia  -  dekantácia  -  dekantácia
l2 mutagénnosť
l2 ionizujúcieť
4701 needs to be added manually:  flokulant  -  flokulant  -  fl

In [16]:
# for manual correcting of words
pprint(words_to_manual_add)

{'aerobiológia': 'aerobiológia',
 'aerácia': 'aerácia',
 'agroekosystém': 'agroekosystém',
 'algicíd': 'algicíd',
 'alkylický': 'alkylická zlúčenina',
 'amoniakálny': 'amoniakálny ión',
 'amonifikácia': 'amonifikácia',
 'antivegetatívny': 'antivegetatívny prostriedok',
 'arídný': 'ekosystém arídnej oblasti',
 'benzopyrén': 'benzopyrén',
 'bezzávlahový': 'bezzávlahové hospodárenie',
 'biofond': 'biologické zdroje biofond',
 'biokoncentrácia': 'faktor biokoncentrácie',
 'biostimulátor': 'rastlinný biostimulátor',
 'chlóretylén': 'chlóretylén',
 'chlórfenol': 'chlórfenol',
 'chlórfluórovaný': 'čiastočne halogenovaný chlórfluórovaný uhľovodík',
 'chlórfluóruhľovodík': 'chlórfluóruhľovodík',
 'chorológia': 'chorológia',
 'dechlorácia': 'dechlorácia',
 'dekantácia': 'dekantácia',
 'dezinfenkčný': 'dezinfenkčný prostriedok',
 'dibenuofurana': 'polychlórovaný dibenuofurán',
 'dibenzodioxina': 'polychlórovaný dibenzodioxín',
 'ekoaudit': 'environmentálne hodnotenie ekoaudit',
 'ekomanažment': '

In [17]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "zlúčenina"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

zlúčenina
model
[('zlúčeniny', 0.8037198781967163),
 ('nitrozlúčeniny', 0.7126240134239197),
 ('halogenid', 0.6617395877838135),
 ('anión', 0.6408039331436157),
 ('katión', 0.6377048492431641),
 ('bróm', 0.6359386444091797),
 ('halogenácia', 0.6336995363235474),
 ('sulfid', 0.6225067377090454),
 ('fosfida', 0.619005560874939),
 ('hydrid', 0.6180623769760132)]
model_lemma_big ['zlúčenín', 'zlúčenina', 'zlúčeninách', 'zlúčeninami', 'zlúčeninu', 'zlúčeninou', 'zlúčeninám', 'zlúčenine', 'Zlúčenina', 'Zlúčeninu', 'zlúčeninach']
model big
[('gama-zlúčenina', 0.7384223341941833),
 ('Zlúčenina', 0.7192478179931641),
 ('zlúčenin', 0.7019988894462585),
 ('zlúčeninu', 0.6797387599945068),
 ('zlúčeninou', 0.6643490195274353),
 ('oxozlúčenina', 0.6469773054122925),
 ('zlúčeniny', 0.6378831267356873),
 ('zlúčeninach', 0.6368275880813599),
 ('zlúčenine', 0.6333630084991455),
 ('spečenina', 0.6204385757446289)]
model big LOWER ['zlúčenina', 'Zlúčenina']
[('gama-zlúčenina', 0.7967488765716553),
 ('Zlúč

In [18]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {'0': 'nula',
 'aerobiológia': 'aero;biológia',
 'aerácia': 'prevzdušňovanie',
 'agroekosystém': 'agro;ekosystém',
 'akvakultúryť': 'akvakultúra',
 'algicíd': 'likvidátor;rias',
 'alkylický': 'alkyl',
 'amoniakálny': 'amoniakový',
 'aox': 'halogenovaný;organický;zlúčenina',
 'aríden': 'suchý',
 'arídný': 'suchý',
 'bezzávlahový': 'bez;závlahový',
 'bioakumulatívny': 'bioakumulatívna',
 'biochemickať': 'biochemický',
 'biodegradabilita': 'biodegradácia',
 'biomonitoring': 'biomonitoring',
 'biostimulátor': 'zvyšovanie;obranyschopnosť;organizmus',
 'chemickať': 'chemický',
 'chloróz': 'chloróza',
 'chlóretylén': 'vinylchlorid',
 'chorológia': 'chorologický',
 'co2': 'co2',
 'dechlorácia': 'odstrániť;chlór',
 'dekantácia': 'oddelovať;usadenina',
 'dezinfenkčný': 'dezinfekčný',
 'dibenuofurana': 'dibenzofurán',
 'doktrínať': 'doktrína',
 'domácať': 'domáci',
 'domácnostiť': 'domácnosť',
 'ekoaudit': 'ekologický;audit',
 'ekomanažment': 'ekologický;manažment',
 'ekomarketing': 'ekologický;marketing',
 'ekomúzeum': 'ekologický;múzeum',
 'ekopediatrie': 'ekologický;pediatria',
 'ekotoxicita': 'ekologický;toxicita',
 'ekoznačenie': 'ekologický;značenie',
 'ekozón': 'ekologický;zóna',
 'elektrokinetika': 'pohyb;elektrický;náboj',
 'elektroodpad': 'elektroodpad',
 'flokulant': 'vločkovač',
 'fytotoxicita': 'rastlinný;toxicita',
 'gasohol': 'benzín;metanol',
 'genotoxicita': 'mutagén',
 'havárijný': 'havarijný',
 'hovädziť': 'hovädzí',
 'imisný': 'imisie',
 'jednogeneračný': 'jedno;generačný',
 'kompostovateľný': 'kompostovateľný',
 'korodovateľnosť': 'korodovať',
 'krmný': 'krmná',
 'legislativo': 'legislatíva',
 'lisovny': 'lisovňa',
 'litorálna': 'litorálna',
 'makrofyta': 'makrofyty',
 'medzipalubný': 'medzi;palubný',
 'metainformačný': 'metainformačný',
 'mikroekosystém': 'mikro;ekosystém',
 'mikrofiltrácia': 'mikro;filtrácia',
 'mikroklimatológia': 'mikro;klimatológia',
 'mikropolutant': 'mikro;polutant',
 'mimosezónny': 'mimo;sezóna',
 'mokraďný': 'mokraď',
 'multispektrálny': 'multi;spektrálny',
 'máloprodukčný': 'málo;produkcia',
 'nakladnie': 'nakladanie',
 'neznečisťujúk': 'znečisťujúci',
 'obstáravanie': 'obstarávanie',
 'odfarbovaca': 'odfarbovať',
 'odlivovať': 'odliv',
 'odpadovať': 'odpadový',
 'odvetvedieť': 'odvetvie',
 'parcelovanie': 'parcela',
 'peý': 'peší',
 'polietavý': 'lietavý',
 'posypovať': 'posypový',
 'použitať': 'použitý',
 'povedomieť': 'povedomie',
 'povrchovať': 'povrchový',
 'pozemkovať': 'pozemkový',
 'poľnohospodárstvať': 'poľnohospodárstvo',
 'prevencať': 'prevencia',
 'prevlhčený': 'navlhčený',
 'priemysieť': 'priemysel',
 'priesakovať': 'priesakový',
 'prokaryotný': 'prokaryot',
 'protierózny': 'protierózna',
 'prádnuť': 'pranie',
 'prístreý': 'prístrešok',
 'pôsť': 'pôda',
 'rozhodovaciť': 'rozhodovací',
 'rozpoznávaciť': 'rozpoznávací',
 'rádioaktívný': 'rádioaktívny',
 'rýchlotest': 'rýchlotest',
 'socioekonómia': 'socioekonomický',
 'spotry': 'spotreba',
 'syndróť': 'syndróm',
 'sídieť': 'sídlo',
 'tepnúť': 'teplo',
 'terasovaný': 'terasa',
 'umelať': 'umelý',
 'urbánnúť': 'urbánne',
 'usporiadanúť': 'usporiadanie',
 'vládnať': 'mimovládna',
 'vonkajý': 'vonkajší',
 'vymiznutý': 'vymiznutý',
 'vypaľovaniť': 'vypaľovanie',
 'vyrubovaniť': 'vyrubovanie',
 'vytavovanie': 'tavenie',
 'vzdelávaciť': 'vzdelávací',
 'výmladkový': 'výmladka',
 'zadržiavaciť': 'zadržiavací',
 'zasoľovanúť': 'zasoľovanie',
 'zbrať': 'zber',
 'zmierňovacia': 'zmierňovať',
 'zmluvať': 'zmluva',
 'znečisťujúcieť': 'znečisťujúci',
 'zneškodniteľný': 'zneškodniť',
 'úrovny': 'úroveň',
 'ťažbať': 'ťažba',
 'želežný': 'železný'}
    
len(new_words_manually_checked)

120

In [19]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [20]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)

0 0
1 aerobiológia
2 aerácia
3 agroekosystém
4 akvakultúryť
5 algicíd
6 alkylický
7 amoniakálny
8 aox
9 aríden
10 arídný
11 bezzávlahový
12 bioakumulatívny
13 biochemickať
14 biodegradabilita
15 biomonitoring
16 biostimulátor
17 chemickať
18 chloróz
19 chlóretylén
20 chorológia
21 co2
22 dechlorácia
23 dekantácia
24 dezinfenkčný
25 dibenuofurana
26 doktrínať
27 domácať
28 domácnostiť
29 ekoaudit
30 ekomanažment
31 ekomarketing
32 ekomúzeum
33 ekopediatrie
34 ekotoxicita
35 ekoznačenie
36 ekozón
37 elektrokinetika
38 elektroodpad
39 flokulant
40 fytotoxicita
41 gasohol
42 genotoxicita
43 havárijný
44 hovädziť
45 imisný
46 jednogeneračný
47 kompostovateľný
48 korodovateľnosť
49 krmný
50 legislativo
51 lisovny
52 litorálna
53 makrofyta
54 medzipalubný
55 metainformačný
56 mikroekosystém
57 mikrofiltrácia
58 mikroklimatológia
59 mikropolutant
60 mimosezónny
61 mokraďný
62 multispektrálny
63 máloprodukčný
64 nakladnie
65 neznečisťujúk
66 obstáravanie
67 odfarbovaca
68 odlivovať
69 odpadovať

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


In [21]:
word = 'audit'
print(len(big_lemmas_set), len(big_words_set))
print(word in big_lemmas_set)
word in big_words_set
get_vector(word)

919919 1525977
True


array([-0.0068    , -0.06601429,  0.03737143,  0.00845714,  0.0118    ,
        0.00501429,  0.03164286, -0.03458571, -0.00238571,  0.01258571,
        0.03444286, -0.04475714,  0.0069    , -0.02132857, -0.02857143,
        0.01737143,  0.03714285,  0.00361429,  0.06777143, -0.01192857,
       -0.04972857, -0.006     , -0.03024286,  0.01135714,  0.01075714,
        0.02821429,  0.04435714,  0.0344    ,  0.03007143,  0.04022857,
       -0.04707143, -0.05254286, -0.01332857,  0.04085714, -0.03525714,
        0.07175715, -0.00164286,  0.0166    , -0.01274286, -0.01464286,
       -0.01507143, -0.01828572,  0.02007143,  0.05201429, -0.04522857,
        0.05564285,  0.01948571,  0.00354286,  0.02487143,  0.00718571,
        0.02164286,  0.04497143, -0.00768571,  0.04848571,  0.0553    ,
       -0.0002    ,  0.02652857,  0.0032    ,  0.01345714,  0.0523    ,
       -0.02394286, -0.04664286,  0.01558571,  0.01727143, -0.03675714,
       -0.03588571, -0.04072857, -0.02511428, -0.01587143, -0.03

In [22]:
# add new words to the model
print(len(new_words), print(len(new_vectors)))
model.add(new_words,new_vectors)

120
120 None


In [23]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    topic = topic.strip()
    topic_nolemma = topic_nolemma.strip()
    topic_words = topic.split('_')
    found = 1
    for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            found = 0
            print('nonexistend word: ', word, " - ", topic_nolemma)

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma.strip())
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma.strip())
                    
print(len(excluded_topics)) 

nonexistend word:  salinizácia  -  salinizácia pôdy
!!! topic needs to be excluded:  salinizácia pôdy
nonexistend word:  biofond  -  biologické zdroje (biofond)
!!! topic needs to be excluded:  biologické zdroje (biofond)
nonexistend word:  termoselektívny  -  termoselektívny proces
!!! topic needs to be excluded:  termoselektívny proces
nonexistend word:  reitrondukcia  -  reitrondukcia živočíšnych druhov
!!! topic needs to be excluded:  reitrondukcia živočíšnych druhov
nonexistend word:  pipať  -  end-of-pipe technológia
!!! topic needs to be excluded:  end-of-pipe technológia
nonexistend word:  semiarídny  -  ekosystémy semiarídnych oblastí
!!! topic needs to be excluded:  ekosystémy semiarídnych oblastí
nonexistend word:  salinizácia  -  salinizácia
!!! topic needs to be excluded:  salinizácia
nonexistend word:  sorpcia  -  sorpcia
!!! topic needs to be excluded:  sorpcia
nonexistend word:  intertidálna  -  intertidálna zóna
!!! topic needs to be excluded:  intertidálna zóna
nonexi

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


nonexistend word:  protiropný  -  protiropná záchytná bariéra
!!! topic needs to be excluded:  protiropná záchytná bariéra
nonexistend word:  šagasova  -  Šagasova choroba
!!! topic needs to be excluded:  Šagasova choroba
nonexistend word:  fluidika  -  fluidika
!!! topic needs to be excluded:  fluidika
nonexistend word:  lipofilický  -  lipofilická látka
!!! topic needs to be excluded:  lipofilická látka
nonexistend word:  speňovadlo  -  speňovadlo
!!! topic needs to be excluded:  speňovadlo
nonexistend word:  fytosociológia  -  fytosociológia
!!! topic needs to be excluded:  fytosociológia
nonexistend word:  amonifikácia  -  amonifikácia
!!! topic needs to be excluded:  amonifikácia
nonexistend word:  imunoanalýz  -  imunoanalýza
!!! topic needs to be excluded:  imunoanalýza
nonexistend word:  inertizácia  -  inertizácia
!!! topic needs to be excluded:  inertizácia
nonexistend word:  perchlóretylén  -  perchlóretylén
!!! topic needs to be excluded:  perchlóretylén
nonexistend word:  

In [24]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  59
['salinizácia pôdy',
 'biologické zdroje (biofond)',
 'termoselektívny proces',
 'reitrondukcia živočíšnych druhov',
 'end-of-pipe technológia',
 'ekosystémy semiarídnych oblastí',
 'salinizácia',
 'sorpcia',
 'intertidálna zóna',
 'revegetácia',
 'flokulácia',
 'World Wide Web',
 'sociopedagogická činnosť',
 'Schistocerca gregaria',
 'perifytón',
 'protiropná záchytná bariéra',
 'Šagasova choroba',
 'fluidika',
 'lipofilická látka',
 'speňovadlo',
 'fytosociológia',
 'amonifikácia',
 'imunoanalýza',
 'inertizácia',
 'perchlóretylén',
 'geogénny faktor',
 'lov do tenatových sietí',
 'schistosomiáza',
 'faktor biokoncentrácie',
 'zákaz freónov a halónov',
 'halogénovaný terfenyl',
 'sapróbny index',
 'testovanie na mutagenicitu',
 'chlórfluóruhľovodík',
 'polyetylénterftalát',
 'odkrývka',
 'teratogenicita',
 'fluoridizácia',
 'polychlórovaný terfenyl',
 'skúška na zistenie teratogenézy',
 'antivegetatívny prostriedok',
 'pentachlórfenol',
 'čiastočne halogenovaný c

In [25]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

lokálny rozvoj  ---  lokálny_rozvoj
rozvoj vidieka  ---  rozvoj_vidiek
riadenie (manažment) vodných zdrojov  ---  riadenie_manažment_vodný_zdroj
úroveň vzdelania  ---  úroveň_vzdelanie
všeobecné vzdelávanie  ---  všeobecný_vzdelávanie
násyp  ---  násyp
právo Európskeho spoločenstva  ---  právo_európsky_spoločenstvo
kultúrne dedičstvo  ---  kultúrny_dedičstvo
poľnohospodárska pôda  ---  poľnohospodársko_pôda
poľnohospodárska krajina  ---  poľnohospodársko_krajina
poľnohospodárska produkcia  ---  poľnohospodársko_produkcia
ekonomický rozvoj  ---  ekonomický_rozvoj
potravový reťazec  ---  potravový_reťazec
potravinársky priemysel  ---  potravinárska_priemysel
kvalita potravín  ---  kvalita_potravina
globálne otepľovanie  ---  globálny_otepľovanie
skleníkový efekt  ---  skleníkový_efekt
priemyselný rozvoj  ---  priemyselný_rozvoj
trh práce  ---  trh_práca
prístup na pozemok  ---  prístup_pozemok
komasácia  ---  komasácia
územné plánovanie  ---  územný_plánovať
krajinná architektúra  ---  k

kultúrny vývoj  ---  kultúrny_vývoj
krajina  ---  krajina
poľnohospodárska legislatíva  ---  poľnohospodársko_legislatíva
starostlivosť o zdravie  ---  starostlivosť_zdravie
roztiahnuté mesto  ---  roztiahnutý_mesto
zdroje  ---  zdroj
drenážna voda  ---  drenážna_voda
regionálna štruktúra  ---  regionálny_štruktúra
poľnohospodárska štruktúra  ---  poľnohospodársko_štruktúra
chudobní ľudia  ---  chudobný_ľudia
dopravná infraštruktúra  ---  dopravný_infraštruktúra
bezpečnostný systém  ---  bezpečnostný_systém
infraštruktúra  ---  infraštruktúra
dekontaminácia pôdy  ---  dekontaminácia_pôda
pastvina  ---  pastvina
ekonomický rast  ---  ekonomický_rast
degradácia sladkých vôd  ---  degradácia_sladký_voda
potrava  ---  potrava
acidifikácia pôdy  ---  acidifikácia_pôda
biologické dedičstvo  ---  biologický_dedičstvo
právo Európskeho spoločenstva  ---  právo_európsky_spoločenstvo
hospodárska budova  ---  hospodársko_budova
priemyselná budova  ---  priemyselný_budova
klíma  ---  klíma
kozmetic

úžitková voda  ---  úžitkový_voda
budova  ---  budova
analýza vody  ---  analýza_voda
analýza pôdy  ---  analýza_pôda
vplyv znečistenia (na prostredie)  ---  vplyv_znečistenie_prostredie
degradácia životného prostredia  ---  degradácia_životný_prostredie
fyzikálny proces  ---  fyzikálny_proces
ochrana živočíchov  ---  ochrana_živočích
ochrana živočíchov  ---  ochrana_živočích
ochrana lesa  ---  ochrana_les
záruka kvality  ---  záruka_kvalita
zelené hnojivo  ---  zelený_hnojivo
mestské obytné prostredie  ---  mestský_obytný_prostredie
vzdelávací systém  ---  vzdelávaciť_systém
mestské zariadenie  ---  mestský_zariadenie
tradičná architektúra  ---  tradičný_architektúra
skladba populácie  ---  skladba_populácia
škody spôsobené vodou  ---  škoda_spôsobený_voda
látky znečisťujúce mestské prostredie  ---  látka_znečisťujúci_mestský_prostredie
technologický proces  ---  technologický_proces
veľkosť podniku  ---  veľký_podnik
biologický vývoj  ---  biologický_vývoj
škoda na životnom prostredí

tvorba krajiny  ---  tvorba_krajina
krajinný prvok  ---  krajinný_prvok
právny systém  ---  právny_systém
citlivá oblasť  ---  citlivý_oblasť
harmonizácia práva  ---  harmonizácia_právo
sociálne zabezpečenie  ---  sociálny_zabezpečenie
inštitucionálna štruktúra  ---  inštitucionálny_štruktúra
úkryt živočíchov  ---  úkryt_živočích
odsoľovanie vody  ---  odsoľovanie_voda
verejná budova  ---  verejný_budova
vtáky  ---  vták
dopravné nariadenie  ---  dopravný_nariadenie
dynamika spoločnosti  ---  dynamika_spoločnosť
marketing  ---  marketing
súkromný automobil  ---  súkromný_automobil
poľnohospodárske stroje  ---  poľnohospodársky_stroj
legislatíva týkajúca sa zdravia  ---  legislatíva_týkajúci_zdraviť
banský právny predpis  ---  banský_právny_predpis
odpadová voda  ---  odpadovať_voda
verejné zaobstarávanie  ---  verejný_zaobstarávanie
tovar a služby  ---  tovar_služba
pôdna vrstva  ---  pôdno_vrstva
priemyselná ekonómia  ---  priemyselný_ekonómia
ekológia pôdy  ---  ekológia_pôda
spoloče

environmentálny stimul  ---  environmentálny_stimul
ľudské osídlenie  ---  ľudský_osídlenie
plánovacie opatrenie  ---  plánovacia_opatrenie
správanie živočíchov  ---  správanie_živočích
prieskum trhu  ---  prieskum_trh
pestovanie zeleniny a ovocia na predaj  ---  pestovanie_zeleniny_ovocia_predaj
populácia rastlín  ---  populácia_rastlina
demineralizácia vody  ---  demineralizácia_voda
uplatnenie práva  ---  uplatnenie_právo
implementácia zákona  ---  implementácia_zákon
environmentálna úzkosť  ---  environmentálny_úzkosť
štátna inštitúcia  ---  štátny_inštitúcia
uhoľný priemysel  ---  uhoľný_priemysel
priemyselný kal  ---  priemyselný_kal
spoločnosť  ---  spoločnosť
analýza spoločnosti  ---  analýza_spoločnosť
kultúrne zariadenie  ---  kultúrny_zariadenie
región  ---  región
emisie do vôd  ---  emisie_voda
hospodársky plán  ---  hospodársky_plán
civilná bezpečnosť  ---  civilný_bezpečnosť
pedológia  ---  pedológia
akustická kvalita  ---  akustický_kvalita
environmentálne kritérium  --

...

zmes plynov  ---  zmes_plyn
vzťahy medzi rasami  ---  vzťah_rasa
prúdenie znečisťujúcej látky  ---  prúdenie_znečisťujúci_látka
roztavenie jadra reaktora  ---  roztavenie_jadry_reaktor
prechodné opatrenie  ---  prechodný_opatrenie
nábytok  ---  nábytok
radiátor  ---  radiátor
komparatívny test  ---  komparatívna_test
kľudová forma  ---  kľudový_forma
jadrová reakcia  ---  jadrový_reakcia
veterný mlyn  ---  veterný_mlyn
gaštan  ---  gaštan
rozpustnosť  ---  rozpustnosť
registračná procedúra  ---  registračný_procedúra
riadená skládka  ---  riadený_skládka
vysoká zver  ---  vysoký_zver
štatistické rady  ---  štatistický_rad
=== EXCLUDE  Šagasova choroba
choroby  ---  choroba
endemické druhy  ---  endemický_druh
semenné rastliny  ---  semenný_rastlina
dolina  ---  dolina
stoka  ---  stok
izotopová indikátorová metóda  ---  izotopový_indikátorový_metóda
mýtus  ---  mýtus
alkylická zlúčenina  ---  alkylický_zlúčenina
diferenciácia  ---  diferenciácia
stajňa  ---  stajňa
pec  ---  pec
právo 

zriedená kyselina  ---  zriedený_kyselina
procedúra pojednávania  ---  procedúra_pojednávanie
najvyššia prípustná koncentrácia  ---  vyšý_prípustný_koncentrácia
potok  ---  potok
potrebný súhlas pre inštaláciu zariadenia  ---  potrebný_súhlas_inštalácia_zariadenie
jadrová elektráreň  ---  jadrový_elektráreň
porovnanie produktu  ---  porovnanie_produkt
identifikácia znečisťujúcich látok  ---  identifikácia_znečisťujúci_látok
pastovitý odpad  ---  pastovitý_odpad
informačný zdroj  ---  informačný_zdroj
toxín  ---  toxina
mlyn  ---  mlyn
multispektrálny skener  ---  multispektrálny_skener
resorpcia  ---  resorpcia
získavanie ropy z ropných zvyškov  ---  získavať_ropa_ropný_zvyšok
rozhodnutie  ---  rozhodnutie
poplach  ---  poplo
vykládka dreva  ---  vykládka_drevo
kalkulácia  ---  kalkulácia
rezolúcia  ---  rezolúcia
spaľovací motor  ---  spaľovací_motor
nekovy  ---  nekov
laboratórny test  ---  laboratórna_test
záhrada  ---  záhrada
obchod s farbami a lakmi  ---  obchod_farb_lak
protiprá

zdroj difúzneho znečistenia  ---  zdroj_difúzny_znečistenie
popol  ---  popnuť
chróm  ---  chró
cín  ---  cina
referenčná látka  ---  referenčný_látka
halogénovaný bifenyl  ---  halogénovaný_bifenyl
zbrojenie  ---  zbrojenie
horľavá látka  ---  horľavý_látka
štvortaktný motor  ---  štvortaktný_motor
dvojtaktný motor  ---  dvojtaktný_motor
uhličitan draselný  ---  uhličitan_draselný
nikel  ---  nikel
datovanie  ---  datovanie
embryo  ---  embryo
fotogrametria  ---  fotogrametrie
botanika  ---  botanika
samočistenie  ---  samočistenie
znárodnenie  ---  znárodnenie
exponát  ---  exponata
odstrašujúci prostriedok  ---  odstrašujúci_prostriedok
odstavenie elektrárne z prevádzky  ---  odstavenie_elektráreň_prevádzka
francúzska záhrada  ---  francúzsko_záhrada
jaskyňa  ---  jaskyňa
anión  ---  anión
ľadová návrš  ---  ľadový_návrš
otrava  ---  otrava
asimilácia  ---  asimilácia
náterová látka  ---  náterovo_látka
polyvinylchlorid  ---  polyvinylchlorid
detektor  ---  detektor
riziko  ---  riz

vulkanológia  ---  vulkanológia
lampa  ---  lampa
sklenné vlákno  ---  sklenný_vlákno
berýlium  ---  berýlium
leptanie  ---  leptanie
decibel  ---  decibel
sporák  ---  sporák
amín  ---  amina
geotechnika  ---  geotechnika
feromón  ---  feromón
izotop  ---  izotop
vymiznuté druhy  ---  vymiznutý_druh
atrament  ---  atrament
glaciológia  ---  glaciológia
platina  ---  platina
osievanie  ---  osievanie
zranenie  ---  zranenie
test karcinogenity  ---  test_karcinogenita
spaľovacie zariadenie  ---  spaľovacia_zariadenie
ochranná vrstva  ---  ochranný_vrstva
fenol  ---  fenol
spevavec  ---  spevavec
nevratný obal  ---  nevratný_obal
chorológia  ---  chorológia
mramor  ---  mramor
=== EXCLUDE  teratogenéza
potlačenie  ---  potlačenie
salamandry  ---  salamandra
radón  ---  radón
tesnenie  ---  tesnenie
odsírovanie  ---  odsírovanie
vačkovce  ---  vačkovca
blchy  ---  blcha
zásaditosť  ---  zásaditosť
lyžiarska dráha  ---  lyžiarsko_dráha
=== EXCLUDE  spazmatický záškrt
silo  ---  silo
viskoz

slony  ---  slon
agroturistika  ---  agroturistika
predstavenie  ---  predstavenie
zosuv snehu  ---  zosuv_sneh
hubky  ---  hubka
inventarizácia  ---  inventarizácia
bioetika  ---  bioetika
cievnaté rastliny  ---  cievnatý_rastlina
UNCED  ---  unced
medveďovité  ---  medveďovité
bioreaktor  ---  bioreaktor
biologická bezpečnosť  ---  biologický_bezpečnosť
vrt  ---  vrt
účtovníctvo  ---  účtovníctvo
Excluded:  59
Final topics:  5183 5183


In [26]:
# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i,topic in enumerate(topics):
    #print(i, topic)
    topic = topic.strip()
    topic_words = topic.split('_')
    #if len(topic_words)<2:
    #    continue
    tmp_vectors = []
    for word in topic_words:
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                raise Warning(f"XXX {word} {topic}")
                print("XXX", word, topic) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  lokálny_rozvoj
append  rozvoj_vidiek
append  riadenie_manažment_vodný_zdroj
append  úroveň_vzdelanie
append  všeobecný_vzdelávanie
--- only one word, not a new topic> násyp
append  právo_európsky_spoločenstvo
append  kultúrny_dedičstvo
append  poľnohospodársko_pôda
append  poľnohospodársko_krajina
append  poľnohospodársko_produkcia
append  ekonomický_rozvoj

NOT in vocabulary:  potravový
append  potravový
append  potravový_reťazec
append  potravinárska_priemysel
append  kvalita_potravina
append  globálny_otepľovanie
append  skleníkový_efekt
append  priemyselný_rozvoj
append  trh_práca
append  prístup_pozemok
--- only one word, not a new topic> komasácia
append  územný_plánovať
append  krajinný_architektúra
append  krajinný_plánovať
append  využívanie_pôda_územie
append  plánovať_využiť_krajina
append  životný_prostredie_mora_oceán
append  ochrana_príroda
doubled TOPIC:  ochrana_príroda
append  ekologický_poľnohospodárstvo
append  priestorový_plánovať
append  základný_vzdelanie


append  integrovaný_manažment
append  funkcia_pôda
append  zmiešaný_poľnohospodárstvo
append  mestský_zeleň
append  surový_voda
append  pôvodný_les
append  komunálna_úroveň
append  priemysel_minerál
append  správny_riadenie
append  prístupový_cesta
append  súkromný_právo
append  verejný_právo
append  chémia_pôda
append  fyzikálny_veda
append  náuka_voda
append  populačný_hustota
append  opustený_priemyselný_krajina
append  vládno_politika
append  rozvojový_kooperácia
--- only one word, not a new topic> organizácia
append  štruktúra_priemysel
doubled TOPIC:  ekonomický_systém
append  verejný_kontrakt
append  starý_osoba
append  pobrežný_znečistenie
append  znečistenie_sladký_voda
append  ch_hydina
doubled TOPIC:  úrodnosť_pôda
append  environmentálny_trh
append  vodný_čerpadlo
append  energetický_trh
--- only one word, not a new topic> konzervovanie
append  vodný_hladina
append  poľnohospodársko_budova
append  komerčný_aktivita
append  verejný_práca
append  environmentálny_bezpečnosť
ap

append  obytný_príves
append  zákon_hnojivo
append  kontrola_kvalita
append  púštny_podnebie
append  priemyselný_asociácia

NOT in vocabulary:  živočí
append  voľne_žijúci_živočí
append  pôdna_profil
append  chránený_krajina
append  sociálny_vybavenie
append  zdravotnícka_zariadenie
append  karibský_oblasť
append  oblasť_poskytovaný_služba
append  verejný_konať
append  lekársky_veda
append  spoločenský_veda
append  miestny_pôvodný_technológia
append  verejný_štátny_dlh
append  prefabrikovaný_budova
append  obnova_mesto
append  spoločenský_indikátor
append  ekonomický_región
append  globálny_model
append  obchodný_organizácia
append  environmentálny_psychológia
append  štátny_lesa
append  prírodný_park
append  orgán_miestny_správa
append  spoločenský_hodnota
append  mestský_stres
append  stavebný_komponent
append  ochrana_druh
doubled TOPIC:  morský_voda
append  environmentálne_povedomieť
append  ochrana_voľne_žijúci_organizmus
doubled TOPIC:  ochrana_voľne_žijúci_organizmus
doubled TOP

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


append  národný_park
append  organizácia_práca
append  ekonomický_podnet
--- only one word, not a new topic> poisťovníctvo
append  regionálny_štatistika
append  mierny_podnebie
append  ekologický_inventarizácia
append  presun_informácia
append  štruktúra_podnik
append  spracovaný_potravina
append  zákon_krmivo
append  prírodný_pamiatka
append  národný_hospodárstvo
append  dolný_snemovňa
append  verejný_mienka
append  priemyselný_zónovanie
append  ekonomický_zónovanie
append  priemyselný_emisie

NOT in vocabulary:  rybárenie
append  rybárenie
append  štruktúra_spôsob_rybárenie
append  prírodný_scenéria

NOT in vocabulary:  povinnosť
append  povinnosť
append  občiansky_právo_povinnosť_týkajúci_životný_prostredie
append  dobrovoľný_práca
append  atmosferický_proces
append  prístup_kultúra
append  právo_veda
append  miestny_doprava
append  zmiešaný_les
append  ochrana_podzemný_voda
append  spracovanie_obrazový_dát_obrázok
append  potrava_živočích
append  informačný_technológia
append  kont

...

--- only one word, not a new topic> mladý
append  nepriama_induktor
append  čistenie_les
--- only one word, not a new topic> drenáž
append  medziodborový_výskum
append  sieť_biotopa
--- only one word, not a new topic> rekreácia
append  sedimentačný_nádrž

NOT in vocabulary:  medzinárodne
append  medzinárodne
append  medzinárodne_významný_ekosystém
append  tradičný_kultúra
--- only one word, not a new topic> desorpcia
--- only one word, not a new topic> biotop
append  veterinárna_medicína
append  listnatý_strom

NOT in vocabulary:  evaporácia
append  evaporácia
--- only one word, not a new topic> evaporácia
--- only one word, not a new topic> odrezka
append  legislatívny_konať
append  biologický_cyklus
append  cyklistický_cesta

NOT in vocabulary:  aeróbne
append  aeróbne
append  aeróbne_podmienka
append  slnečný_energia
--- only one word, not a new topic> hornina
append  biologický_kontaminácia
--- only one word, not a new topic> afrika
append  energetický_surovina
--- only one word, n

append  sadovnícko
append  sadovnícko_škôlka
--- only one word, not a new topic> marš

NOT in vocabulary:  ozónový
append  ozónový_vrstva
append  sopečný_erupcia
append  atmosferický_emisie
append  doprava_zamestnanie
--- only one word, not a new topic> stavebníctvo
--- only one word, not a new topic> zabavenie
append  teplota_oceán_mora
append  obchod_spotry
append  čierny_more
append  minerálny_olej
--- only one word, not a new topic> konvencia
append  hlavný_riziko
append  povolanie_zamestnanie
append  tvorba_komplexný_zlúčenina
append  seizmický_činnosť
--- only one word, not a new topic> kanál
--- only one word, not a new topic> poľovníctvo
append  abiotický_faktor
--- only one word, not a new topic> výchova
append  odpadovať_voda_lisovny_olivový_olej
append  umelý_hnojivo
append  zákon_chemikália
append  medzinárodný_obchod
append  stavebný_hluk
append  obchodný_činnosť
append  odpadový_plyn
append  kategória_odpad
--- only one word, not a new topic> predmestie
append  policajný_

append  kombinovaný_znečistenie
append  mapovanie_lišajník
append  dozorný_úrad
--- only one word, not a new topic> úlovka
--- only one word, not a new topic> predátor
--- only one word, not a new topic> trajektória
--- only one word, not a new topic> fungicíd
append  štátny_prírodný_rezervácia
append  meranie_chemický_znečistenie
append  rádioaktívny_odpad
--- only one word, not a new topic> herbicíd
--- only one word, not a new topic> uhľovodík
append  aromatický_uhľovodík
append  mechanika_hornina
append  vojenský_aspekt
--- only one word, not a new topic> geológia
append  rybol_otvorený_more
append  rezervácia_voľne_žijúci_druh
append  spracovanie_odpad
--- only one word, not a new topic> list
append  rekultivácia_ťažobný_územie
--- only one word, not a new topic> disperzia
append  záchranný_plán
append  horský_úkryt
append  mortalita_ryba
append  lodný_nehoda
append  dlhodobý_trend
--- only one word, not a new topic> laser
append  centrálna_tepláreň
--- only one word, not a new to

--- only one word, not a new topic> subjekt
append  čierny_uhlie
append  riziko_rakovina
append  príčina_ochorenie
--- only one word, not a new topic> bór
--- only one word, not a new topic> ramenonožca
append  drevený_uhlie
append  odpad_spracovanie_odpad

NOT in vocabulary:  znečisťujú
append  znečisťujú
append  vystavenie_človek_vplyv_znečisťujú_látok
--- only one word, not a new topic> kniha
append  štrkový_jama
append  prechodný_prvok
--- only one word, not a new topic> farmakológia
append  odpad_záhrada
append  starý_environmentálny_záťaž
--- only one word, not a new topic> vrak
append  analýza_bezpečenstvať

NOT in vocabulary:  navrstvovanie
append  navrstvovanie
append  navrstvovanie_obrazový_dát
--- only one word, not a new topic> vlna

NOT in vocabulary:  ultrafiltrácia
append  ultrafiltrácia
--- only one word, not a new topic> ultrafiltrácia
--- only one word, not a new topic> litosféra
append  zákonný_dokument
append  tepelný_elektráreň
append  poplatok_odpada
append  zákon

append  mačkovité
--- only one word, not a new topic> mačkovité
append  zemepisný_šírka
append  imisný_predpoveď
append  vytýčenie_trasa
append  škoda_spôsobený_vojenský_manévra
--- only one word, not a new topic> centralizácia
--- only one word, not a new topic> povolenie
--- only one word, not a new topic> korytnačka
--- only one word, not a new topic> had
append  komunálny_služba
--- only one word, not a new topic> vektor
append  jemný_pre
--- only one word, not a new topic> zákonník
append  odber_vzorka
--- only one word, not a new topic> benzina
--- only one word, not a new topic> katión

NOT in vocabulary:  adries
append  adries
append  zozna_adries
--- only one word, not a new topic> zoológia
--- only one word, not a new topic> orgán
--- only one word, not a new topic> blanokrídlovca
append  registrácia_látok
--- only one word, not a new topic> koks
append  námorný_mapa
append  bod_var
--- only one word, not a new topic> biorytmus

NOT in vocabulary:  bioklimatológia
append  bio

In [27]:
# add topics to model
model.add(new_words,new_vectors)

In [28]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [29]:
# try the model
model.similar_by_vector('vnútrozemský_plavba')

[('plavba', 0.9378631114959717),
 ('výletný_plavba', 0.9018838405609131),
 ('námorný_plavba', 0.892289936542511),
 ('vnútrozemský', 0.7433691024780273),
 ('okružný_cesta_loď', 0.7240498661994934),
 ('vnútrozemský_vodný_doprava', 0.7171871662139893),
 ('plavie', 0.7141085267066956),
 ('výletný', 0.6961541175842285),
 ('moreplavba', 0.6902751922607422),
 ('morský_pobrežie', 0.6860016584396362)]

In [30]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 lokálny_rozvoj
0 [('regionálny_rozvoj', 0.9107511043548584), ('ekonomický_rozvoj', 0.8762646913528442), ('technologický_rozvoj', 0.8750103712081909), ('priemyselný_rozvoj', 0.8743530511856079), ('lokálny', 0.844261646270752), ('rozvoj', 0.8420019149780273), ('environmentálny_rozvoj_vývoj', 0.8415120840072632), ('ekologický_rozvoj_vývoj', 0.8308059573173523), ('lokálny_zalesňovanie', 0.8276740312576294), ('rozvoj_spoločnosť', 0.819735050201416)]

-----
 rozvoj_vidiek
1 [('vidiek', 0.8517659902572632), ('osídlenie_vidiek', 0.8247206807136536), ('rozvoj', 0.8133547306060791), ('rozvoj_mesto', 0.8058696985244751), ('ekonomický_rozvoj', 0.8039273023605347), ('populácia_vidiek', 0.8037147521972656), ('priemyselný_rozvoj', 0.7996429204940796), ('regionálny_rozvoj', 0.7945408821105957), ('rozvoj_spoločnosť', 0.7789438962936401), ('dokument_rozvoj_mesto', 0.7738286852836609)]

-----
 riadenie_manažment_vodný_zdroj
2 [('rozvoj_vodný_zdroj', 0.9128099679946899), ('legislatíva_vodný_zdroj'

29 [('ekologický_hospodárstvo', 0.9457281827926636), ('ekologický', 0.8958412408828735), ('ekologický_spoločenstvo', 0.8350223302841187), ('poľnohospodársko_ekológia', 0.8128495216369629), ('ekologický_katastrofa', 0.7948930859565735), ('poľnohospodárstvo', 0.7917866110801697), ('poľnohospodárstvať', 0.7917866110801697), ('ekomanažment', 0.789581298828125), ('ekologický_civilizácia', 0.7877241969108582), ('ekonomicko_ekologický_efektívnosť', 0.7864755392074585)]

-----
 priestorový_plánovať
30 [('priemyselný_plánovať', 0.8628804683685303), ('poľnohospodársky_plánovať', 0.8514446020126343), ('hospodársky_plánovať', 0.8491595387458801), ('environmentálny_plánovať', 0.8437060117721558), ('regionálny_plánovať', 0.8424531817436218), ('plánovať', 0.8346233367919922), ('krajinný_plánovať', 0.7988386154174805), ('priestorový', 0.7977455854415894), ('stavebný_plánovať', 0.7904897928237915), ('vzdelávacia_plánovať', 0.7846004962921143)]

-----
 základný_vzdelanie
31 [('vzdelanie', 0.909989356994

59 [('územný_plánovať', 0.9339622259140015), ('územný_plán', 0.8703739047050476), ('územný', 0.8653238415718079), ('centrálny_riadenie', 0.8020036816596985), ('regionálny_plánovať', 0.7995177507400513), ('hospodársky_plánovať', 0.7940994501113892), ('správny_riadenie', 0.7821692824363708), ('plán_riadenie', 0.7748420834541321), ('environmentálny_plánovať', 0.7737529277801514), ('prostriedok_ekonomický_riadenie', 0.7708073854446411)]

-----
 rozvoj_vodný_zdroj
60 [('vodný_zdroj', 0.9309231638908386), ('legislatíva_vodný_zdroj', 0.9232627749443054), ('ochrana_vodný_zdroj', 0.9198033213615417), ('riadenie_manažment_vodný_zdroj', 0.9128099679946899), ('sanácia_ozdravenie_vodný_zdroj', 0.8958588242530823), ('vodný_hospodárstvo', 0.8590706586837769), ('vodný_prostredie', 0.8585541248321533), ('vodný_staviteľstvo', 0.8488325476646423), ('hospodárstvo_spoločný_využívanie_zdroj', 0.8456910252571106), ('využívanie_miestny_zdroj', 0.844810962677002)]

-----
 priemyselný_plodina
61 [('plodina', 0.

86 [('zdravotnícky', 0.7446779608726501), ('zdravotnícka', 0.7287362813949585), ('zdravotník', 0.7284705638885498), ('školstvo', 0.7089270949363708), ('sociálny_medicína', 0.7041910886764526), ('zdravotnícka_zariadenie', 0.7040095329284668), ('organizácia_školstvo', 0.6922693252563477), ('zdravotný', 0.6741092205047607), ('zdravotná', 0.6735987663269043), ('zdravotno', 0.6592267751693726)]

-----
 sociálny_služba
87 [('verejný_služba', 0.9047485589981079), ('informačný_služba', 0.8934444189071655), ('služba', 0.8779183626174927), ('sociálny_zabezpečenie', 0.8693146705627441), ('sociálny_starostlivosť', 0.8623169660568237), ('sociálny', 0.8557677268981934), ('komunálny_služba', 0.8497512340545654), ('poradenský_služba', 0.8443346619606018), ('referenčný_služba', 0.8439871072769165), ('oblasť_poskytovaný_služba', 0.8394033312797546)]

-----
 prírodný_zdroj
88 [('manažment_prírodný_zdroj', 0.9423844814300537), ('ochrana_prírodný_zdroj', 0.9188613295555115), ('degradácia_prírodný_zdroj', 0

112 [('stabilizácia_pôda', 0.8821030855178833), ('máloprodukčný_pôda', 0.8694376945495605), ('hospodárenie_pôda', 0.8681846261024475), ('acidifikácia_pôda', 0.8675384521484375), ('pôda', 0.8644355535507202), ('pôsť', 0.8644355535507202), ('usadzovanie_pôda', 0.8600931167602539), ('zaťaženie_pôda', 0.8596717119216919), ('poľnohospodársko_pôda', 0.8582639098167419), ('dekontaminácia_pôda', 0.8567132949829102)]

-----
 demografický_evolúcia
113 [('evolúcia', 0.8498217463493347), ('demografický_vývoj', 0.806775689125061), ('demografický', 0.7873265147209167), ('biologický_vývoj', 0.7430524826049805), ('evolučný', 0.7340152859687805), ('demograficko', 0.7323815822601318), ('demografia', 0.7220100164413452), ('genetický_premena', 0.7143781185150146), ('environmentálny_aspekt_ľudský_osídľovanie', 0.7108951807022095), ('demografick', 0.7052996158599854)]

-----
 odborný_školenie
114 [('environmentálny_školenie', 0.9065667390823364), ('školenie', 0.9037468433380127), ('odborný', 0.8533480167388

...

5055 [('ekosystéma', 0.8529071807861328), ('degradácia_ekosystéma', 0.8264729976654053), ('hranica_ekosystéma', 0.7750160694122314), ('hodnotný_zložka_ekosystéma', 0.771496057510376), ('zraniteľnosť_ekosystém', 0.770845353603363), ('ekosystém', 0.7673462629318237), ('interakcia_ekosystém', 0.7409032583236694), ('skúmať_ekosystéma', 0.7362427115440369), ('mokraďný_ekosystém', 0.7318353652954102), ('ekosystémový', 0.7283517122268677)]

-----
 chemický_odpad_podnik_obchod
5056 [('odpad_obchodný_činnosť', 0.8881372213363647), ('chemický_odpad', 0.8744449615478516), ('priemyselný_odpad', 0.8592696189880371), ('zvláštny_priemyselný_odpad', 0.8506695032119751), ('farmaceutický_odpad', 0.8462467193603516), ('spotrebiteľský_odpad', 0.8381533622741699), ('nakladanie_odpad_priemyselný_oblasť', 0.8366309404373169), ('odpad_potravina', 0.8352786302566528), ('chemický_úprava_odpad', 0.8347181081771851), ('závod_spracovanie_odpad', 0.831809401512146)]

-----
 dôsledok_zmeny_klím
5057 [('politika_obla

5084 [('eúť', 0.757351279258728), ('systém_obchodovanie_emisia', 0.6540290117263794), ('systém_obchodovanie_emisný_kvóta', 0.6357977390289307), ('kompenzačný_politika_obchodovanie_emisia', 0.6145576238632202), ('emisia', 0.5801585912704468), ('kontrola_emisia', 0.577933669090271), ('zdroj_emisia', 0.5542360544204712), ('znižovanie_emisia', 0.5457471609115601), ('emisný_kvóta', 0.5447404384613037), ('register_emisia', 0.5391719937324524)]

-----
 európsky_environmentálna_informačný_pozorovateľský_sieť
5085 [('environmentálna_informačný_sieť', 0.9513511657714844), ('elektronický_informačný_sieť', 0.8926346898078918), ('informačný_sieť', 0.8863168954849243), ('európsky_agentúra_životný_prostredie', 0.8504178524017334), ('geografický_informačný_systém', 0.8441929817199707), ('informačný_infraštruktúra', 0.8430439233779907), ('národný_environmentálny_účtovníctvo', 0.8361064791679382), ('finančný_prostriedok_európsky_spoločenstvo', 0.8282071352005005), ('národný_program_ochrana_životný_prost

5114 [('metóda_environmentálny_stopa_výrobok', 0.9423354268074036), ('environmentálny_stopa', 0.8557911515235901), ('ekologický_stopa', 0.82430100440979), ('biochemický_metóda', 0.7943915128707886), ('matematický_metóda', 0.7940008640289307), ('poľnohospodársko_metóda', 0.7923105359077454), ('interpretačný_metóda', 0.7846808433532715), ('metóda_hodnotenie', 0.7816194295883179), ('analytický_metóda', 0.7716265916824341), ('metóda', 0.7686987519264221)]

-----
 poškodzovanie_ozón
5115 [('poškodenie_stratosférický_ozón', 0.8965490460395813), ('ozón', 0.8862313032150269), ('atmosferický_ozón', 0.8488735556602478), ('troposférický_ozón', 0.8047308921813965), ('potenciál_poškodzovanie_ozónový_vrstva', 0.7551021575927734), ('poškodenie_ozónový_vrstva', 0.7120552062988281), ('ozónový', 0.6772330403327942), ('poškodzovanie', 0.6635168194770813), ('ozónový_vrstva', 0.653831958770752), ('ohrozenie_znečisťujúci_látok', 0.6521236896514893)]

-----
 odpad_park
5116 [('odpad_záhrada', 0.8230020403862

5144 [('potravinárska_priemysel', 0.9424459934234619), ('drevospracujúci_priemysel', 0.8709504008293152), ('environmentálny_priemysel', 0.8669658303260803), ('priemysieť', 0.8655378818511963), ('priemysel', 0.8655378818511963), ('gumárenský_priemysel', 0.8651951551437378), ('elektrotechnický_priemysel', 0.8553677797317505), ('cukrovarnícka_priemysel', 0.8511017560958862), ('nábytkárska_priemysel', 0.8480609059333801), ('petrochemický_priemysel', 0.8422778844833374)]

-----
 agrometeorológia
5145 [('hydrometeorológia', 0.8946841359138489), ('meteorológia', 0.8168705701828003), ('hydrometeorologický', 0.700567364692688), ('hydrometeorologicko', 0.6930780410766602), ('meteorológ', 0.6848205327987671), ('hydrológia', 0.6537679433822632), ('virológia', 0.6339420080184937), ('synekológia', 0.6208302974700928), ('paleoekológia', 0.6146366596221924), ('klimatológia', 0.612594723701477)]

-----
 ekologický_rozvoj_vývoj
5146 [('environmentálny_rozvoj_vývoj', 0.9826292991638184), ('technologický_

5173 [('špongia', 0.7124593257904053), ('leštička', 0.6718761920928955), ('kefka', 0.6664673089981079), ('pinzeta', 0.648459255695343), ('štetinka', 0.6442331671714783), ('kefa', 0.6346731185913086), ('utierka', 0.6339049935340881), ('kožtička', 0.6328597068786621), ('metlička', 0.6308932900428772), ('štetina', 0.6300038695335388)]

-----
 inventarizácia
5174 [('ekologický_inventarizácia', 0.8468952178955078), ('inventarizačný', 0.7551782131195068), ('inventúra', 0.6881406307220459), ('účtovná_evidencia', 0.6063679456710815), ('sumarizácia', 0.601120114326477), ('arizácia', 0.5913045406341553), ('inventarizácia_škoda_lesy', 0.5691765546798706), ('evidencia', 0.5443600416183472), ('ekoaudit', 0.5246073007583618), ('inventár', 0.5229541063308716)]

-----
 bioetika
5175 [('environmentálny_etika', 0.7088300585746765), ('noetika', 0.6818354725837708), ('etika', 0.680199921131134), ('metaetika', 0.6435604691505432), ('problematika_environmentálny_ekonómia', 0.6351226568222046), ('environment